In [1]:
import math
import numpy as np
import tensorflow as tf

import evaluate

In [2]:
def conv_block(x, n_filters, kernel_size=3):
    x = tf.keras.layers.Conv1D(n_filters, kernel_size, padding='same', activation='relu')(x)
    return x

def downsample_block(x, pool_size=4):
    x = tf.keras.layers.MaxPool1D(pool_size)(x)
    return x

def get_model_CMV1(n_filters=64, kernel_size=5):
    # CMNet V1
    model_input = tf.keras.layers.Input(shape=(4000,2))

    x = model_input
    for i in range(5):
        x = conv_block(x, n_filters, kernel_size)
        x = downsample_block(x)

    x = tf.keras.layers.Conv1D(1, 3, padding='valid', activation='linear')(x)

    model = tf.keras.Model(model_input, x)
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=tf.keras.metrics.RootMeanSquaredError())
    return model

print('CMV2-5')
get_model_CMV1(kernel_size=5).summary()
print('CMV2-7')
get_model_CMV1(kernel_size=7).summary()
print('CMV2-9')
get_model_CMV1(kernel_size=9).summary()

CMV2-5
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4000, 2)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 4000, 64)          704       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1000, 64)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 64)          20544     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 250, 64)           20544     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 62, 64)           

In [3]:
for model in [get_model_CMV1(kernel_size=5), get_model_CMV1(kernel_size=7), get_model_CMV1(kernel_size=9)]:
    Wsave = model.get_weights()
    for dataset in ['BIDMC32HR', 'BIDMC32RR', 'BIDMC32SpO2']:
        mse_list = []
        rmse_list = []

        train_x = np.load(f'dataset/{dataset}_train_x.npy')
        train_y = np.load(f'dataset/{dataset}_train_y.npy')
        test_x  = np.load(f'dataset/{dataset}_test_x.npy')
        test_y  = np.load(f'dataset/{dataset}_test_y.npy')
        train_x = np.swapaxes(train_x, 1, 2)
        test_x = np.swapaxes(test_x, 1, 2)
        
        for i in range(5):
            model.set_weights(Wsave)

            # model selection
            # model = get_model_CMV1()
            model.fit(x=train_x, y=train_y, batch_size=32, epochs=128, verbose=0)
            mse, rmse = model.evaluate(test_x, test_y)
            mse_list.append(mse)
            rmse_list.append(rmse)

        print(dataset)
        print('MEAN')
        print('MSE :', np.mean(mse_list))
        print('RMSE:', np.mean(rmse_list))

75/75 [==============================] - 0s 5ms/step - loss: 69.5906 - root_mean_squared_error: 8.3421
BIDMC32HR
MEAN
MSE : 74.52982482910156
RMSE: 8.572298908233643
75/75 [==============================] - 0s 5ms/step - loss: 36.2962 - root_mean_squared_error: 6.0246
BIDMC32RR
MEAN
MSE : 38.85454063415527
RMSE: 6.194613265991211
75/75 [==============================] - 0s 6ms/step - loss: 33.9255 - root_mean_squared_error: 5.8246
BIDMC32SpO2
MEAN
MSE : 29.755582809448242
RMSE: 5.438367748260498
75/75 [==============================] - 0s 5ms/step - loss: 85.1391 - root_mean_squared_error: 9.2271
BIDMC32HR
MEAN
MSE : 73.8404655456543
RMSE: 8.564058208465577
75/75 [==============================] - 0s 5ms/step - loss: 28.6856 - root_mean_squared_error: 5.3559
BIDMC32RR
MEAN
MSE : 34.25733642578125
RMSE: 5.833164405822754
75/75 [==============================] - 0s 4ms/step - loss: 34.6670 - root_mean_squared_error: 5.8879
BIDMC32SpO2
MEAN
MSE : 29.67038230895996
RMSE: 5.413960266113281
